In [45]:
# 많은 문장 혹은 문서들 중 유사한 문장을 찾아내는 방법
# 어떤 문장을 벡터로 표현할 수 있다면 벡터 간 거리를 구하는 방법으로 손쉽게 해결
from sklearn.feature_extraction.text import CountVectorizer

In [46]:
vectorizer = CountVectorizer(min_df = 1)

In [47]:
contents = ['메리랑 놀러가고 싶지만 바쁜데 어떻하죠?',
           '메리는 공원에서 산책하고 노는 것을 싫어해요',
           '메리는 공원에서 노는 것도 싫어해요. 이상해요.',
           '먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요']

In [48]:
X = vectorizer.fit_transform(contents)
vectorizer.get_feature_names()

['것도',
 '것을',
 '곳으로',
 '공원에서',
 '그러질',
 '너무',
 '노는',
 '놀러가고',
 '떠나고',
 '메리는',
 '메리랑',
 '못하고',
 '바빠서',
 '바쁜데',
 '산책하고',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떻하죠',
 '여행을',
 '이상해요',
 '있어요']

In [49]:
from konlpy.tag import Okt
okt = Okt()

In [50]:
contents_tokens = [okt.morphs(row) for row in contents]
contents_tokens

[['메리', '랑', '놀러', '가고', '싶지만', '바쁜데', '어떻하죠', '?'],
 ['메리', '는', '공원', '에서', '산책', '하고', '노', '는', '것', '을', '싫어해요'],
 ['메리', '는', '공원', '에서', '노', '는', '것', '도', '싫어해요', '.', '이상해요', '.'],
 ['먼',
  '곳',
  '으로',
  '여행',
  '을',
  '떠나고',
  '싶은데',
  '너무',
  '바빠서',
  '그러질',
  '못',
  '하고',
  '있어요']]

In [51]:
# 형태소 분석을 한 후 띄어쓰기로 구분하고 그것 자체를 하나의 문장으로 만들어서
# scikit learn의 vectorizer 함수에서 사용하기 편하게 편집
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 메리 랑 놀러 가고 싶지만 바쁜데 어떻하죠 ?',
 ' 메리 는 공원 에서 산책 하고 노 는 것 을 싫어해요',
 ' 메리 는 공원 에서 노 는 것 도 싫어해요 . 이상해요 .',
 ' 먼 곳 으로 여행 을 떠나고 싶은데 너무 바빠서 그러질 못 하고 있어요']

In [52]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(4, 20)

In [53]:
vectorizer.get_feature_names()

['가고',
 '공원',
 '그러질',
 '너무',
 '놀러',
 '떠나고',
 '메리',
 '바빠서',
 '바쁜데',
 '산책',
 '싫어해요',
 '싶은데',
 '싶지만',
 '어떻하죠',
 '에서',
 '여행',
 '으로',
 '이상해요',
 '있어요',
 '하고']

In [54]:
X.toarray().transpose()

array([[1, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 1, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1]])

In [55]:
# 각 벡터들 사이의 거리 구하기
new_post = ['메리랑 공원에서 산책하고 놀고 싶어요']
new_post_tokens = [okt.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 메리 랑 공원 에서 산책 하고 놀고 싶어요']

In [56]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]])

In [57]:
import scipy as sp
def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [58]:
best_doc = None
best_dist = 65535
best_i = None

for i in range(0, num_samples):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)
    print("== Post %i with dist=%.2f : %s" %(i, d, contents[i]))
    if d < best_dist:
        best_dist = d
        best_i =i

== Post 0 with dist=3.00 : 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
== Post 1 with dist=1.00 : 메리는 공원에서 산책하고 노는 것을 싫어해요
== Post 2 with dist=2.00 : 메리는 공원에서 노는 것도 싫어해요. 이상해요.
== Post 3 with dist=3.46 : 먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요


In [59]:
print("Best post is %i, dist = %.2f" % (best_i, best_dist))
print("-->", new_post)
print("----->", contents[best_i])

Best post is 1, dist = 1.00
--> ['메리랑 공원에서 산책하고 놀고 싶어요']
-----> 메리는 공원에서 산책하고 노는 것을 싫어해요


In [60]:
for i in range(0, len(contents)):
    print(X.getrow(i).toarray())
print('---------------------')
print(new_post_vec.toarray())

[[1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0]]
[[0 1 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1]]
[[0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0]]
[[0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 0 1 1]]
---------------------
[[0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1]]


In [61]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    delta = v1_normalized - v2_normalized
    return sp.linalg.norm(delta.toarray())

In [62]:
best_doc = None
best_dist = 66535
best_i = None

for i in range(0, num_samples):
    post_vec = X.getrow(i)
    d = dist_norm(post_vec, new_post_vec)
    
    print("== Post %i with dist = %.2f  : %s" %(i, d, contents[i]))
    
    if d < best_dist:
        best_dist = d
        best_i = i

== Post 0 with dist = 1.28  : 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
== Post 1 with dist = 0.42  : 메리는 공원에서 산책하고 노는 것을 싫어해요
== Post 2 with dist = 0.89  : 메리는 공원에서 노는 것도 싫어해요. 이상해요.
== Post 3 with dist = 1.30  : 먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요


In [63]:
print("Best post is %i, dist = %.2f" % (best_i, best_dist))
print("-->", new_post)
print("----->", contents[best_i])

Best post is 1, dist = 0.42
--> ['메리랑 공원에서 산책하고 놀고 싶어요']
-----> 메리는 공원에서 산책하고 노는 것을 싫어해요


In [77]:
# tfidf
# tf(term frequency)
# idf(inverse document frequency)
# 텍스트 마이닝에서 사용하는 일종의 단어별로 부과하는 가중치.
# tf는 어떤 단어가 문서 내에서 자주 등장할수록 중요도가 높을 것으로 보는 것
# idf는 비교하는 모든 문서에 만약 같은 단어가 있다면 이 단어는 핵심 어휘일지는 모르지만
# 문서 간의 비교에서는 중요한 단어가 아니라는 뜻으로 보는 것
from numpy.lib import scimath
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = scimath.log(float(len(D))/(len([doc for doc in D if t in doc])))
    return tf, idf

In [78]:
a, abb, abc = ['a'], ['a', 'b', 'b'], ['a', 'b', 'c']
D = [a, abb, abc]

print(tfidf('a', a, D))
print(tfidf('b', abb, D))
print(tfidf('a', abc, D))
print(tfidf('b', abc, D))
print(tfidf('c', abc, D))

(1.0, 0.0)
(0.6666666666666666, 0.4054651081081644)
(0.3333333333333333, 0.0)
(0.3333333333333333, 0.4054651081081644)
(0.3333333333333333, 1.0986122886681098)


In [79]:
# 두 값을 곱한 것을 tfidf라고 하는 함수로 수정해서 사용하면 되지만 scikit-learn의 TfidfVectorizer를 이용
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')

In [80]:
contents_tokens = [okt.morphs(row) for row in contents]

contents_for_vectorize = []
for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    contents_for_vectorize.append(sentence)
    
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(4, 20)

In [82]:
vectorizer.get_feature_names_out()

array(['가고', '공원', '그러질', '너무', '놀러', '떠나고', '메리', '바빠서', '바쁜데', '산책',
       '싫어해요', '싶은데', '싶지만', '어떻하죠', '에서', '여행', '으로', '이상해요', '있어요',
       '하고'], dtype=object)

In [83]:
new_post = ['근처 공원에 메리랑 놀러가고 싶네요.']
new_post_tokens = [okt.morphs(row) for row in new_post]
new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)
new_post_for_vectorize

[' 근처 공원 에 메리 랑 놀러 가고 싶네요 .']

In [84]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)

In [87]:
best_doc = None
best_dist = 65535
best_i = None

for i in range(0, num_samples):
    post_vec = X.getrow(i)
    d = dist_norm(post_vec, new_post_vec)
    
    print("== Post %i with dist= %.2f  : %s" % (i, d, contents[i]))
    
    if d < best_dist:
        best_dist = d
        best_i = i

print("Best post is %i , dist = %.2f" % (best_i, best_dist))
print('-->', new_post)
print('----->', contents[best_i])

== Post 0 with dist= 0.90  : 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
== Post 1 with dist= 1.18  : 메리는 공원에서 산책하고 노는 것을 싫어해요
== Post 2 with dist= 1.16  : 메리는 공원에서 노는 것도 싫어해요. 이상해요.
== Post 3 with dist= 1.41  : 먼 곳으로 여행을 떠나고 싶은데 너무 바빠서 그러질 못하고 있어요
Best post is 0 , dist = 0.90
--> ['근처 공원에 메리랑 놀러가고 싶네요.']
-----> 메리랑 놀러가고 싶지만 바쁜데 어떻하죠?
